# Introducción al Aprendizaje Federado.


### ¿Qué es el Aprendizaje Federado?

Es una forma simple y poderosa de entrenar modelos. Si se piensa en los datos de entrenamiento,siempre son el resultado de algún tipo de proceso de recopilación. Las personas (a través de dispositivos) generan datos al registrar eventos en el mundo real. Normalmente, estos datos se agregan a una única ubicación central para que pueda entrenar un modelo de aprendizaje automático.

En lugar de llevar los datos de entrenamiento al modelo (un servidor central), traes el modelo a los datos de entrenamiento (donde sea que se encuentren).

La idea es que esto permite que quienquiera que esté creando los datos posea la única copia permanente, y así mantener el control sobre quién tiene acceso a ella.

# Section 2.1 - A Toy Federated Learning Example

Para comprender exactamente que es el aprendizaje federado vamos a ver las diferencias entre esta técnica y la que se emplea habitualmente.

Vamos a empezar entrenando un modelo de forma centralizada. 
Necesitamos:

- Un dataset de prueba (Toy Dataset)
- Un modelo
- Lógica de entrenamiento

In [1]:
import syft as sy
hook = sy.TorchHook()
from torch import nn, optim

# A Toy Dataset
data = sy.Var(sy.FloatTensor([[0,0],[0,1],[1,0],[1,1]]))
target = sy.Var(sy.FloatTensor([[0],[0],[1],[1]]))

# A Toy Model
model = nn.Linear(2,1)

def train():
    # Training Logic
    opt = optim.SGD(params=model.parameters(),lr=0.1)
    for iter in range(20):

        # 1) erase previous gradients (if they exist)
        opt.zero_grad()

        # 2) make a prediction
        pred = model(data)

        # 3) calculate how much the missed
        loss = ((pred - target)**2).sum()

        # 4) figure out which weights caused us to miss
        loss.backward()

        # 5) change those weights
        opt.step()

        # 6) print our progress
        print(loss.data[0])

In [2]:
train()

3.076326608657837
1.768141746520996
1.2177542448043823
0.8632838129997253
0.6180337071418762
0.4457544684410095
0.3236871063709259
0.23652707040309906
0.17383378744125366
0.1284254640340805
0.09532319754362106
0.07104768604040146
0.053148481994867325
0.03988617658615112
0.03001665510237217
0.022643668577075005
0.017117079347372055
0.01296229474246502
0.009830850176513195
0.007465502247214317


Hemos entrenado un modelo básico de la manera convencional. Todos nuestros datos se agregan a nuestra máquina local y podemos usarlos para realizar actualizaciones en nuestro modelo. El aprendizaje federado, sin embargo, no funciona de esta manera. Entonces, modifiquemos este ejemplo para realizarlo con aprendizaje federado.

Necesitamos:

- crear un par de trabajadores.
- Obtener los punteros para entrenar los datos en cada trabajador.
- Lógica de entrenamiento actualizada para realizar aprendizaje federado. 

    Nuevos pasos de entrenamiento.
    - Enviar el modelo al trabajador correcto.
    - Entrenar con los datos localizados en ese trabajador.
    - Traer de vuelta el modelo y repetir con el siguiente trabajador.

In [3]:
# create a couple workers

bob = sy.VirtualWorker(id="bob")
alice = sy.VirtualWorker(id="alice")

# get pointers to training data on each worker by
# sending some training data to bob and alice
data_bob = data[0:2].send(bob)
target_bob = target[0:2].send(bob)

data_alice = data[2:].send(alice)
target_alice = target[2:].send(alice)

# organize pointers into a list
datasets = [(data_bob,target_bob),(data_alice,target_alice)]

# Iniitalize A Toy Model
model = nn.Linear(2,1)

def train():
    # Training Logic
    opt = optim.SGD(params=model.parameters(),lr=0.1)
    for iter in range(20):
        
        # NEW) iterate through each worker's dataset
        for data,target in datasets:
            
            # NEW) send model to correct worker
            model.send(data.location)

            # 1) erase previous gradients (if they exist)
            opt.zero_grad()

            # 2) make a prediction
            pred = model(data)

            # 3) calculate how much the missed
            loss = ((pred - target)**2).sum()

            # 4) figure out which weights caused us to miss
            loss.backward()

            # NEW) get model (with gradients)
            model.get()

            # 5) change those weights
            opt.step()

            # 6) print our progress
            print(loss.get().data[0]) # NEW) slight edit... need to call .get() on loss

In [4]:
train()

0.24732357263565063
0.35474807024002075
0.295735239982605
0.1121668815612793
0.1884697675704956
0.06399760395288467
0.11686219274997711
0.038787536323070526
0.07299670577049255
0.023917512968182564
0.046079330146312714
0.014988611452281475
0.02942308597266674
0.009561199694871902
0.0190159622579813
0.006215935572981834
0.012444151565432549
0.004120749421417713
0.008246853947639465
0.002784881740808487
0.005533907562494278
0.0019166868878528476
0.003758638398721814
0.0013412302359938622
0.0025824003387242556
0.0009523084736429155
0.001793382689356804
0.0006845876341685653
0.001257741474546492
0.0004971838206984103
0.0008899428066797554
0.0003640666254796088
0.0006346915615722537
0.0002683281491044909
0.00045581170707009733
0.00019875857105944306
0.0003293381887488067
0.0001477920013712719
0.00023921113461256027
0.00011020559759344906


## ¡FIN!

Hemos entrenado un modelo de Aprendizaje Profundo muy simple utilizando el aprendizaje federado. Enviamos el modelo a cada trabajador, generamos un nuevo gradiente y luego devolvemos el gradiente a nuestro servidor local donde actualizamos nuestro modelo global. Nunca en este proceso hemos visto o solicitado acceso a los datos de entrenamiento subyacentes.Preservamos la privacidad de Bob y Alice.

## Deficiencias de este ejemplo

Entonces, aunque este ejemplo es una buena introducción al aprendizaje federado, todavía tiene algunas deficiencias importantes. En particular, cuando llamamos `model.get ()` y recibimos el modelo actualizado de Bob o Alice, podemos realmente aprender mucho sobre los datos de entrenamiento de Bob y Alice mirando sus gradientes. En algunos casos, podemos restaurar sus datos de entrenamiento perfectamente.

¿Entonces, que hay que hacer? Bueno, la primera estrategia que se emplea es promediar el gradiente entre varias personas antes de cargarlo en el servidor central.
